In [3]:
from collections import OrderedDict
from robustness import datasets
from robustness.tools.breeds_helpers import make_living17
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from trainer_functions.living17trainer import build_resnet50, evaluate_living17, train_living17

ImportError: cannot import name 'transforms' from 'torchvision' (unknown location)

In [9]:
# empty dataframe
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

data_dir = '/Users/rada/.mxnet/datasets/imagenet'
info_dir = '/Users/rada/Documents/GitHub/BREEDS-Benchmarks/imagenet_class_hierarchy/modified'

n_subset = 850
lrs = [0.0005, 1e-4, 1e-5]

In [10]:
for lr in lrs:
    
    ret = make_living17(info_dir, split="rand")
    superclasses, subclass_split, label_map = ret
    train_subclasses, test_subclasses = subclass_split

    dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
    loaders_source = dataset_source.make_loaders(workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=850)
    train_loader_source, val_loader_source = loaders_source

    dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
    loaders_target = dataset_target.make_loaders(workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=850)
    train_loader_target, val_loader_target = loaders_target

    device = torch.device("mps")
    tune_net = build_resnet50(device)

    # tune_net.layer1 to tune the first layer, tune_net to tune all layers, tune_net.fc to tune the last year
    # add linear probing here
    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_living17(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_living17(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    optimizer = optim.Adam(tune_net.layer1.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_living17(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "first layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Living17_layer1.csv")


==> Preparing dataset custom_imagenet..
==> Preparing dataset custom_imagenet..


RuntimeError: Expected one of cpu, cuda, mkldnn, opengl, opencl, ideep, hip, msnpu device type at start of device string: mps